### Initialization

Import of standard and third party libraries.

In [1]:
from pathlib import Path
import sys

import astroid
import owlready2 as owl

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Adding `codeontology` to sys path for its import.

In [2]:
codeontology_path = Path("../../../../").resolve()
assert codeontology_path.exists()
sys.path.insert(0, str(codeontology_path))

Loading the ontology.

In [3]:
ontology_path = Path("../../../ontology/codeontology.owl").resolve()
assert ontology_path.exists()
ontology = owl.get_ontology(str(ontology_path)).load()
ontology.base_iri = r"http://rdf.webofcode.org/woc/"

### Code sample

In [4]:
node = astroid.extract_node("""
var: dict[str, tuple[str, unknown] | None] = {'': None}
""")
ann_node = node.annotation

### Obtaining a structured annotation

We start calling the annotation solver to obtain, from an annotation, the node references to the classes in a structured format.

In [5]:
from codeontology.rdfization.python3.extract.transformer.tracking import resolve_annotation
structured_ann = resolve_annotation(ann_node)

In [6]:
structured_ann

(<ClassDef.dict l.0 at 0x13922991880>,
 <ClassDef.str l.0 at 0x13922bab670>,
 [(<ClassDef.tuple l.0 at 0x13922c0ae80>,
   <ClassDef.str l.0 at 0x13922bab670>,
   None),
  <ClassDef.NoneType l.0 at 0x13922ca7250>])

We read this as a `dict` parameterized by: `str`; a `tuple` (couple) of `str` and `unknown`, or `None`. Since `unknown` is actually an unknown name, the structured annotation has a `None` object instead of a class reference on its position.

### Creating `Parameterized Type`s individuals

Here we have, according to the ontology, a type `dict`, that is a `Class` (primitive types, the so called built-in types, are actually classes). Since this `dict` is parameterized, it will be represented with a `Parameterized Type` individual, that will be linked to the original `Class` (`dict`) by the `has generic type` property.

Every actual parameterization (`str` and `tuple[str, unknown] | None`) will be represented by `Type Argument`s individual, that will be linked to the `Parameterized Type` owner (`dict`) by the `is actual type argument of` property, and to the type they represent (`str` and `tuple[str, unknown] | None`) by the `has argument type` property. In the case of `tuple[str, unknown] | None` the property will link to `tuple[str, unknown]` and `None` separately, since they are two ammissable different types (thus, the types are not assumed to be univocal in the ontology).

The need to use an intermediary individual (`Type Argument`) instead of linking directly the `dict` to `str` and `tuple[str, unknown] | None` is given by the fact that we also have to store the information about the position of the type in the parameterization: we first have `str` and then `tuple[str, unknown] | None`, they are not interchangeable. The `Type Argument` is the holder of this information by the `has type argument position` property.

Last thing to say is that a `Type Argument` can be mirroring a `Class`, like for `str`, but also recursively a `Parameterized Type`, like for `tuple[str, unknown]`.

In [7]:
from codeontology.rdfization.python3.extract.extractor import extract_structured_type
param_type = extract_structured_type(structured_ann)

Lets see what has been extracted! The `Parameterized Type` is:

In [8]:
print(f"{param_type.iri} (type {type(param_type).__name__})")
print(f"Has generic type: {param_type.hasGenericType.hasSimpleName}")

http://rdf.webofcode.org/woc/parameterizedtype2 (type ParameterizedType)
Has generic type: dict


Its `Type Argument`s are:

In [9]:
for i, type_arg in enumerate(param_type.hasActualTypeArgument):
    print()
    print(f"TypeArgument #{i}")
    print(f" Actual type argument of: {type_arg.isActualTypeArgumentOf.iri}")
    print(f" Has argument type: {[t.hasSimpleName if type(t) is ontology.Class else t.iri for t in type_arg.hasArgumentType]}")


TypeArgument #0
 Actual type argument of: http://rdf.webofcode.org/woc/parameterizedtype2
 Has argument type: ['str']

TypeArgument #1
 Actual type argument of: http://rdf.webofcode.org/woc/parameterizedtype2
 Has argument type: ['http://rdf.webofcode.org/woc/parameterizedtype1', 'NoneType']


Lets check the `Parameterized Type` behind the second `Type Argument`:

In [10]:
param_type_nested = param_type.hasActualTypeArgument[1].hasArgumentType[0]

print(f"{param_type_nested.iri} (type {type(param_type_nested).__name__})")
print(f"Has generic type: {param_type_nested.hasGenericType.hasSimpleName}")

for i, type_arg in enumerate(param_type_nested.hasActualTypeArgument):
    print()
    print(f"TypeArgument #{i}")
    print(f" Actual type argument of: {type_arg.isActualTypeArgumentOf.iri}")
    print(f" Has argument type: {[t.hasSimpleName if type(t) is ontology.Class else t.iri for t in type_arg.hasArgumentType]}")

http://rdf.webofcode.org/woc/parameterizedtype1 (type ParameterizedType)
Has generic type: tuple

TypeArgument #0
 Actual type argument of: http://rdf.webofcode.org/woc/parameterizedtype1
 Has argument type: ['str']

TypeArgument #1
 Actual type argument of: http://rdf.webofcode.org/woc/parameterizedtype1
 Has argument type: []


We can see how the `Type Argument` for the `unknown` type has been created, but there is no link to the type individual itself because we were unable to resolve the AST reference for it (actually, it was impossible, since it does not exist statically!).